In [1]:
# !yes | pip uninstall keras tensorflow tensorflow-estimator tensorflow-hub tensorflow-io-gcs-filesystem

Found existing installation: keras 2.6.0
Uninstalling keras-2.6.0:
  Would remove:
    /usr/local/anaconda3/envs/toewalking/lib/python3.9/site-packages/keras-2.6.0.dist-info/*
    /usr/local/anaconda3/envs/toewalking/lib/python3.9/site-packages/keras/*
Proceed (Y/n)?   Successfully uninstalled keras-2.6.0
Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Would remove:
    /usr/local/anaconda3/envs/toewalking/bin/estimator_ckpt_converter
    /usr/local/anaconda3/envs/toewalking/bin/import_pb_to_tensorboard
    /usr/local/anaconda3/envs/toewalking/bin/saved_model_cli
    /usr/local/anaconda3/envs/toewalking/bin/tensorboard
    /usr/local/anaconda3/envs/toewalking/bin/tf_upgrade_v2
    /usr/local/anaconda3/envs/toewalking/bin/tflite_convert
    /usr/local/anaconda3/envs/toewalking/bin/toco
    /usr/local/anaconda3/envs/toewalking/bin/toco_from_protos
    /usr/local/anaconda3/envs/toewalking/lib/python3.9/site-packages/tensorflow-2.6.0.dist-info/*
    /usr/loca

In [2]:
# %%capture
# # installations
# # !pip install tensorflow 
# !pip install tensorflow==2.7.0
# !pip install tensorflow_hub
# !apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

# !pip install matplotlib
# !pip install ipynb
# !pip install --upgrade PIL

In [1]:
import os
import time
import datetime

import numpy as np
import pandas as pd

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
#import tensorflow_hub as hub
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import glob
from misc_mason import *

import shutil

%load_ext tensorboard
%reload_ext tensorboard
%matplotlib inline

2024-12-18 18:03:33.123531: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734573813.143199   34930 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734573813.149077   34930 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-18 18:03:33.168916: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
folder_path = '/data/datasets/AD/raw_data/'
file_extension = 'nii'
details = list_files(folder_path, file_extension)
print(details['non-nii']['file_paths'])
details['all_file_paths']

['/data/datasets/AD/raw_data/ADNI1_Complete_3Yr_1.5T_11_14_2024.csv', '/data/datasets/AD/raw_data/ADNI1_Screening 1.5T.zip', '/data/datasets/AD/raw_data/ADNI1_Screening_1.5T/ADNI1_Screening_1.5T_6_23_2023.csv', '/data/datasets/AD/raw_data/ADNI1_Screening_1.5T/ADNI1_Screening_1.5T_metadata.zip', '/data/datasets/AD/raw_data/ADNI1_Complete_3Yr_1.5T/ADNI1_Complete_3Yr_1.5T_metadata.zip', '/data/datasets/AD/raw_data/ADNI1_Complete_3Yr_1.5T/ADNI1_Complete_3Yr_1.5T_7_21_2023.csv', '/data/datasets/AD/raw_data/ADNI1_Complete_3Yr_1.5T/ADNI1_Complete 3Yr 1.5T.zip']


['/data/datasets/AD/raw_data/ADNI1_Screening_1.5T/ADNI/131_S_1301/MPR__GradWarp__B1_Correction__N3__Scaled/2007-02-21_11_33_23.0/I63793/ADNI_131_S_1301_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070801190324220_S26899_I63793.nii',
 '/data/datasets/AD/raw_data/ADNI1_Screening_1.5T/ADNI/067_S_0045/MPR-R__GradWarp__B1_Correction__N3__Scaled/2005-11-16_09_58_19.0/I35888/ADNI_067_S_0045_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20070111004845291_S10185_I35888.nii',
 '/data/datasets/AD/raw_data/ADNI1_Screening_1.5T/ADNI/067_S_0045/MPR-R__GradWarp__B1_Correction__N3__Scaled_2/2005-11-16_09_58_19.0/I119184/ADNI_067_S_0045_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_2_Br_20081003155416966_S10185_I119184.nii',
 '/data/datasets/AD/raw_data/ADNI1_Screening_1.5T/ADNI/022_S_0129/MPR__GradWarp__B1_Correction__N3__Scaled/2006-02-06_11_54_32.0/I59484/ADNI_022_S_0129_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070712152611314_S11485_I59484.nii',
 '/data/datasets/AD/raw_data/ADNI1_Scr

In [3]:
def organize_images(source_dir, output_dir):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Iterate through the source directory and its subdirectories
    for root, dirs, files in os.walk(source_dir):
        # Exclude the .ipynb_checkpoints directory
        if '.ipynb_checkpoints' in dirs:
            dirs.remove('.ipynb_checkpoints')

        # Iterate through the files
        for file in files:
            file_path = os.path.join(root, file)

            if 'AD' in file:
                class_label = 'AD'
            elif 'MCI' in file:
                class_label = 'MCI'
            elif 'CN' in file:
                class_label = 'CN'

            # Create the class directory in the output directory if it doesn't exist
            class_dir = os.path.join(output_dir, class_label)
            os.makedirs(class_dir, exist_ok=True)

            # Move the file to the class directory
            dst_path = os.path.join(class_dir, file)
            shutil.move(file_path, dst_path)

    print("Image organization completed.")

In [ ]:
# Convert NII to PNG and create the ADNI_IMG_32.5%_x_unprocessed folder
folder_path = '/data/datasets/AD/raw_data/'
details = list_files(folder_path, 'nii')
all_files = details['all_file_paths']

enum_len = len(list(enumerate(all_files)))

for i, file in enumerate(all_files):
    i += 1
    if '.nii' in file:
        print(file)
        try:
            convert_nii_to_png(file, 0.325, 'x')
            print(f"Done with ({i}/{enum_len})")
        except KeyError:
            # Get the filename without extension
            filename, _ = os.path.splitext(os.path.basename(file))

            # Split the filename by underscores and get the last part
            filename.split('_')[-1]

            print(filename, "NOT found in .csv, skipping!")

print(("-" * 25) + "\nDone with creating folder")

/data/datasets/AD/raw_data/ADNI1_Screening_1.5T/ADNI/131_S_1301/MPR__GradWarp__B1_Correction__N3__Scaled/2007-02-21_11_33_23.0/I63793/ADNI_131_S_1301_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070801190324220_S26899_I63793.nii
/data/datasets/AD/raw_data/ADNI1_Screening_1.5T/ADNI/131_S_1301/MPR__GradWarp__B1_Correction__N3__Scaled/2007-02-21_11_33_23.0/I63793/ADNI_131_S_1301_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070801190324220_S26899_I63793.nii
image_data_id I63793
Image resizing completed.
Done with (1/3257)
/data/datasets/AD/raw_data/ADNI1_Screening_1.5T/ADNI/067_S_0045/MPR-R__GradWarp__B1_Correction__N3__Scaled/2005-11-16_09_58_19.0/I35888/ADNI_067_S_0045_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20070111004845291_S10185_I35888.nii
/data/datasets/AD/raw_data/ADNI1_Screening_1.5T/ADNI/067_S_0045/MPR-R__GradWarp__B1_Correction__N3__Scaled/2005-11-16_09_58_19.0/I35888/ADNI_067_S_0045_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20070111004845291_S10185_I3588

In [4]:
# Organize the PNG images into classes
source_dir = 'ADNI_IMG_32.5%_x'
output_dir = 'ADNI_IMG_32.5%_x_organized'
organize_images(source_dir, output_dir)

Image organization completed.


In [5]:
df = pd.read_csv('/data/datasets/AD/raw_data/ADNI1_Complete_3Yr_1.5T/ADNI1_Complete_3Yr_1.5T_7_21_2023.csv')
{i for i in df.loc[df['Image Data ID'] == "I63793"].Group}.pop()

'CN'

In [6]:
df.loc[df['Image Data ID'] == "I63793"].Group

198    CN
Name: Group, dtype: object

In [7]:
folder_path_ad = 'ADNI_IMG_32.5%_x_organized/AD'
folder_path_mci = 'ADNI_IMG_32.5%_x_organized/MCI'
folder_path_cn = 'ADNI_IMG_32.5%_x_organized/CN'
file_count_ad = len([f for f in os.listdir(folder_path_ad) if os.path.isfile(os.path.join(folder_path_ad, f))])
file_count_mci = len([f for f in os.listdir(folder_path_mci) if os.path.isfile(os.path.join(folder_path_mci, f))])
file_count_cn = len([f for f in os.listdir(folder_path_cn) if os.path.isfile(os.path.join(folder_path_cn, f))])

print(f"Number of files in the folder (AD): {file_count_ad}")
print(f"Number of files in the folder (MCI): {file_count_mci}")
print(f"Number of files in the folder (CN): {file_count_cn}")

Number of files in the folder (AD): 453
Number of files in the folder (MCI): 981
Number of files in the folder (CN): 748


In [8]:
pwd

'/home/mason/ADNI_Dataset'

In [9]:
convert_nii_to_png('/data/datasets/AD/ADNI1_Complete_3Yr_1.5T/ADNI',0.325,'x')

/data/datasets/AD/ADNI1_Complete_3Yr_1.5T/ADNI


IndexError: list index out of range

In [18]:
ls

AD Classification (AD vs. NC).ipynb  AD Progression Prediction.ipynb
AD Classification-Copy1.ipynb        BuildFolders.ipynb
AD Classification.ipynb              misc.py
ADNI_IMG_32.5%_x_organized/          __pycache__/


In [19]:
os.getcwd()

'/home/wen/AD_disease/SURF_2023'